In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Spam Classfication**

In [50]:
# training set
train_set = pd.read_csv('../input/korea-university-cose362-2020-2/train.csv')
train_set.head()

,id,label,message
0,0,0,Hey tmr maybe can meet you at yck
1,1,0,Smith waste da.i wanna gayle.
2,2,1,Well done ENGLAND! Get the official poly ringt...
3,3,0,Just finished. Missing you plenty
4,4,0,U too...


In [51]:
X_train = train_set['message']
y_train = train_set['label']

In [52]:
# test set
test_set = pd.read_csv('../input/korea-university-cose362-2020-2/leaderboard_test_file.csv')
test_set.head()

,id,message
0,0,"SPJanuary Male Sale! Hot Gay chat now cheaper,..."
1,1,have * good weekend.
2,2,Hope this text meets you smiling. If not then ...
3,3,Please tell me you have some of that special s...
4,4,He said i look pretty wif long hair wat. But i...


In [53]:
X_test = test_set['message']

**Tokenizing Data & Coverting into Sequences**

In [54]:
max_words = 1000

token = Tokenizer(num_words = max_words)
token.fit_on_texts(X_train)

sequences = token.texts_to_sequences(X_train)

In [55]:
word_to_index = token.word_index
num_words_size = len(word_to_index) + 1 # + padding 0
print(num_words_size)

7755


In [56]:
print('message max length: %d' % max(len(l) for l in sequences)) # max_len
print('message avg length: %f' % (sum(map(len, sequences))/len(sequences)))

message max length: 171
message avg length: 12.614250


In [57]:
max_len = max(len(l) for l in sequences)
              
sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)

sequences_matrix.shape

(4000, 171)

**Define the Structure of Model**

In [64]:
model = Sequential()
model.add(Embedding(max_words,100,input_length=max_len))
model.add(LSTM(64))
model.add(Dense(256,name='hidden_layer', activation='relu')) # hidden layer
model.add(Dropout(0.2))
model.add(Dense(1,name='output_layer', activation='sigmoid')) # output layer

model.summary()

model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 171, 100)          100000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                42240     
_________________________________________________________________
hidden_layer (Dense)         (None, 256)               16640     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
output_layer (Dense)         (None, 1)                 257       
Total params: 159,137
Trainable params: 159,137
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.fit(sequences_matrix,y_train,batch_size=128,epochs=6,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/6
25/25 [==============================] - 5s 210ms/step - loss: 0.2967 - accuracy: 0.8906 - val_loss: 0.1404 - val_accuracy: 0.9625
Epoch 2/6
25/25 [==============================] - 5s 200ms/step - loss: 0.0738 - accuracy: 0.9809 - val_loss: 0.0722 - val_accuracy: 0.9800
Epoch 3/6
25/25 [==============================] - 5s 191ms/step - loss: 0.0443 - accuracy: 0.9866 - val_loss: 0.0649 - val_accuracy: 0.9812
Epoch 4/6
25/25 [==============================] - 5s 197ms/step - loss: 0.0330 - accuracy: 0.9903 - val_loss: 0.0570 - val_accuracy: 0.9850
Epoch 5/6
25/25 [==============================] - 5s 207ms/step - loss: 0.0253 - accuracy: 0.9931 - val_loss: 0.0613 - val_accuracy: 0.9850


**Prediction**

In [60]:
test_sequences = token.texts_to_sequences(X_test)

#test_max_len = max(len(l) for l in test_sequences)

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

test_sequences_matrix.shape

(300, 171)

In [61]:
y_pred = model.predict_classes(test_sequences_matrix)

print('Predicted value: ',y_pred)

Predicted value:  [[1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [

**Result**

In [62]:
result = test_set.drop(columns=['message'])

result['label'] = y_pred

result.head()

,id,label
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0


In [63]:
result.to_csv('./spamLabelResult2.csv', index = False)

leaderboard score: 0.94117